In [1]:
import os
import re
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from facenet_pytorch import InceptionResnetV1
from PIL import Image


c:\Users\magma\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class FaceDataset(Dataset):
    def __init__(self, root_dir, transform=None, max_classes=10):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []
        self.class_to_idx = {}
        self.idx_to_class = {}
        label_names = []

        for fname in sorted(os.listdir(root_dir)):
            if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
                name_without_ext = fname.rsplit('.', 1)[0]
                parts = name_without_ext.split('@')
                result = '@'.join(parts[:2])
                if result not in label_names:
                    label_names.append(result)

        label_names = label_names[:max_classes]
        self.class_to_idx = {name: idx for idx, name in enumerate(label_names)}
        self.idx_to_class = {idx: name for name, idx in self.class_to_idx.items()}

        for fname in sorted(os.listdir(root_dir)):
            if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
                name_without_ext = fname.rsplit('.', 1)[0]
                parts = name_without_ext.split('@')
                label = '@'.join(parts[:2])
                if label in self.class_to_idx:
                    path = os.path.join(root_dir, fname)
                    self.samples.append((path, self.class_to_idx[label]))
        self.labels = label_names

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label


In [3]:
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
])

# limit do 5 klas - długo zajmuje trening :c
dataset = FaceDataset(root_dir='data/', transform=transform, max_classes=5)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [4]:
model = InceptionResnetV1(pretrained='vggface2', classify=True, num_classes=len(dataset.class_to_idx))

for param in model.parameters():
    param.requires_grad = False

for param in model.logits.parameters():
    param.requires_grad = True
for param in model.last_linear.parameters():
    param.requires_grad = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [5]:
dataset.labels

['101506130@N03_identity_0',
 '102962858@N03_identity_2',
 '105391338@N08_identity_0',
 '105546346@N08_identity_2',
 '105700383@N05_identity_7']

In [6]:
len(dataset.samples)

56

In [7]:
for epoch in range(3):
    total_loss = 0
    correct = 0
    model.train()
    for imgs, labels in dataloader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = outputs.argmax(1)
        correct += (preds == labels).sum().item()

    accuracy = correct / len(dataset)
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, Accuracy: {accuracy:.4f}")

Epoch 1, Loss: 4.5342, Accuracy: 0.5179
Epoch 2, Loss: 0.7377, Accuracy: 0.9643
Epoch 3, Loss: 0.5012, Accuracy: 0.9821


In [8]:
torch.save(model.state_dict(), 'facenet_classifier.pth')


In [ ]:
model.load_state_dict(torch.load('facenet_classifier.pth'))
model.eval()

def predict(image_path, model, transform, class_to_idx):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image)
        pred = output.argmax(1).item()
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    return idx_to_class[pred]

predicted_label = predict('data/101506130@N03_identity_0@10902564096_0.jpg', model, transform, dataset.class_to_idx)
print(f'Predicted label: {predicted_label}')
